In [3]:
!pip install git+https://github.com/openai/swarm.git

  Cloning https://github.com/openai/swarm.git to /tmp/pip-req-build-e_h3v6pg
  Running command git clone --filter=blob:none --quiet https://github.com/openai/swarm.git /tmp/pip-req-build-e_h3v6pg
  Resolved https://github.com/openai/swarm.git to commit 9db581cecaacea0d46a933d6453c312b034dbf47
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.4/69.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.7/218.7 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.0/99.0 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 38.8 MB/s eta 0:00:00
  Created wheel for swarm: filename=swarm-0.1.0-py3-none-any.whl size=26001 sha256=f38a13ac5a4

In [4]:
import os
os.environ["OPENAI_API_KEY"]="OPENAI_API_KEY"

In [5]:
##sample basic demo of swarm

In [6]:
from swarm import Swarm,Agent
client=Swarm()

agent=Agent(
    name="Basic Agent",
    instructions="You are a helpful agent"
)

messages=[{"role":"user","content":"Hi,How are you?"}]
response=client.run(agent=agent,messages=messages)
print(response.messages[-1]["content"])

Hello! I'm here and ready to help. How can I assist you today?


In [7]:
###Transfer work between agents

In [8]:
from swarm import Swarm,Agent
client=Swarm()

english_agent=Agent(
    name="English Agent",
    instructions="You only speak English"
)

french_agent=Agent(
    name="French Agent",
    instructions="You only speak French"
)

def transfer_to_french_agent():
  return french_agent

english_agent.functions.append(transfer_to_french_agent)

messages=[{"role":"user","content":"Hi,How are you?"}]
response=client.run(agent=english_agent,messages=messages)
print(response.messages[-1]["content"])



I'm great, thank you! How can I assist you today?


In [9]:
messages=[{"role":"user","content":"Comment vas-tu?"}]
response=client.run(agent=english_agent,messages=messages)
print(response.messages[-1]["content"])

Je vais bien, merci ! Comment puis-je t'aider aujourd'hui ?


In [10]:
###Context variable within an agent

In [11]:
from swarm import Swarm, Agent

client = Swarm()

def instructions(context_variable):
  name=context_variable.get("name","User")
  return f"You are a helpful agent. Greet the user by name ({name})."

def print_account_details(context_variables:dict):
    user_id = context_variables.get("user_id", None)
    name = context_variables.get("name", None)
    print(f"Account Details: {name} {user_id}")
    return "Success"

def print_salary_details(context_variables:dict):

    salary=context_variables.get("salary", None)
    print(f"Salary is {salary}")
    return "Success"

agent=Agent(
    name="Agent",
    instructions=instructions,
    functions=[print_account_details,print_salary_details],
)

context_variable={"name":"Anjali","user_id":123,"salary":50000}

response=client.run(
    agent=agent,
    messages=[{"role":"user","content":"Hi How are you?"}],
    context_variables=context_variable
)
print(response.messages[-1]["content"])

Hello Anjali! I'm doing well, thank you for asking. How can I assist you today?


In [12]:
response=client.run(
    agent=agent,
    messages=[{"role":"user","content":"print my salary"}],
    context_variables=context_variable
)
print(response.messages[-1]["content"])

Salary is 50000
Your salary details have been successfully retrieved. If there's anything else you need, feel free to ask!


In [13]:
###Function calling

In [14]:
from swarm import Swarm, Agent

client = Swarm()


def get_weather(location) -> str:
    return "{'temp':67, 'unit':'F'}"


agent = Agent(
    name="Agent",
    instructions="You are a helpful agent.",
    functions=[get_weather],
)

messages = [{"role": "user", "content": "What's the weather in NYC?"}]

response = client.run(agent=agent, messages=messages)
print(response.messages[-1]["content"])

The current temperature in New York City is 67°F.
